## Description

Here, we would like to understand how different parameters affect the shape of the PSF. Also, PSD of the object

#### Package required for AMIRAL: 
- numpy
- matplotlib
- astropy
- maoppy --> but I need to think how to implement it because it is being set a bit differently
- decovbench --> 
- cython 

To implement the environment, import the environment from .yml file. (Check to see if it is the most-up-to-date version.)

In [ ]:
# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
#Change to your path
os.chdir("/Users/alau/Repo/amiral")
from amiral import instructment, utils, parameter, gradient, minimisation, array
from scipy.optimize import minimize 
%matplotlib inline

In [ ]:
# Global variable
# Parameters to modify
FLUX = 5e8         # Object total intensity [e-]
test_data_dir  = "/Users/alau/Data/amiral_fits/VESTA/"
image_name = "image_noise_43.fits"

In [ ]:
# move these functions into the plotting (except create psfao19 otf)

def create_psfao19_otf (otf_tel, guess, aosys_cls): 
    
    # Use PSFAO19 model to create a PSF
    psd_ao = aosys_cls.psd_residual_ao (guess = guess)
    psd_halo = aosys_cls.psd_residual_halo(r0 = guess[0])
    
    psd = psd_ao + psd_halo

    otf_atmo = aosys_cls.otf_atmo(psd)
    otf_total = otf_atmo*otf_tel
    
    
    return otf_atmo,otf_total


def plot_otf_total(aosys_cls,otf_total): 
    
    fig, ax = plt.subplots()
    ycent = int((256*aosys_cls.samp_factor[0])//2)

    ax.plot(np.abs(otf_total[ycent,...]))
    ax.set_title('OTF(total)')
    ax.axhline(y=1, color = 'r', ls = '--')
    
    return 0 

def plot_psd_object(): 
    
    fig, ax = plt.subplots()
    ycent = int((256*aosys_cls.samp_factor[0])//2)

    ax.plot(np.abs(otf_total[ycent,...]))
    ax.set_title('OTF(total)')
    ax.axhline(y=1, color = 'r', ls = '--')
    
    pass

In [ ]:
img = utils.load_fits(test_data_dir+image_name)

In [ ]:
aosys_dict = {
    'diameter': 7 , 
    'occ_ratio': 0.1 , 
    'no_acutuator' : 30, 
    'wavelength': 500, 
    'dimension': 256,
    'resolution_rad' : 3.5e-8 
}

In [ ]:
amiral_dict = {
    "r0": 0.1,  #0.2                
    "background": 0.01,      
    "amplitude": 0.1,  #1.6     
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

In [ ]:
psf_keys, psf_guess = utils.dict2array(amiral_dict)

## Generate a PSF 

aoSystem is used to provide a serveral outputs: PSD_array, pupil function (or the pupil plane), and ? - see the IDL output first! aoSystem inherits functions and methods from telescopeSetup (so I wont need to define all parameters again). 

#### Zero-Padding

- remember to pad before fft 
- zoom to area of interest after fft

First, we need to choose an image for deconvolution. In here, we have picked the asteriods. 

Consider changing the true asteriod into PSF $\circledast$ true object. 

#### Image formation theory
\begin{equation}
I = H*O+N, 
\end{equation}

where $I$ is the image, $H$ is the PSF, $O$ is the object and $N$ is the noise.


In the Fourier space,
\begin{equation}
    \hat{I} = \hat{O} \hat{H} + \hat{N}, 
\end{equation}
where $\hat{I}$, $\hat{O}$ and $\hat{N}$ are the Fourier transform of the image, object and noise respectively. Using the fourier space, it is easier for us to calculate the observed image, as we can take an inverse transform of $\hat{O} \hat{H} + \hat{N}$. 

In [ ]:
aosys_cls = instructment.aoSystem( 
        diameter = aosys_dict['diameter'], 
        occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], 
        wavelength = aosys_dict['wavelength']*1e-9, 
        resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension'])  

#### Pupil Function
To get the telescope component of the PSF, we need to know the pupil function first. From the pupil function, auto-correlation function of pupil function will give you the diffraction-limited OTF.

In [ ]:
pupil = aosys_cls.get_pupil_plane()
plt.imshow(pupil)

In [ ]:
# As the functions requires 2D array, for 0 phase offset, 0*pupil_plane will keep the shape
# As you may know the whole otf is in the form of : h = h_tel + h_AO + h_shift (if we need one)
otf_tel = aosys_cls.pupil_to_otf_tel(pupil)

fig, ax =  plt.subplots()
ax.set_title("OTF (diffraction limited)")
pos = ax.imshow(np.real(otf_tel), cmap='Reds', interpolation='none')
fig.colorbar(pos, ax=ax)

#### OTF and PSF

Optical Transfer Function (OTF) is a complex-valued function describing the response of an imaging system as a function of spatial frequency. It is formally defined as the Fourier Transform of the PSF, 
\begin{equation}
\label{eqn:OTF}
    \tilde{H} = \tilde{h}_T.\tilde{h}_\mathrm{{atmo}},
\end{equation}
where $\tilde{H}$ is the total OTF, $\tilde{h}_T$ is the instrument OTF and $\tilde{h}_\mathrm{{atmo}}$ is the residual atmospheric OTF. The instrument OTF can be derived from the pupil function auto-correlation.

In [ ]:
psf_tel = np.fft.fftshift(np.real(utils.ifft2D(otf_tel)))

cx = len(psf_tel[0])/2
print(cx)

fig, ax =  plt.subplots()
ax.set_title("PSF(diffraction limited)")
pos = ax.imshow(np.log10(psf_tel[int(cx)-80:int(cx)+80,int(cx)-80:int(cx)+80]), interpolation='none')
fig.colorbar(pos, ax=ax)

## Fetick-2019-PSFmodel


The aim of this model is not a full PSF reconstruction but to get a physical model to demonstrate it. From \cite{Goodman1968} and \cite{Roddier1981}, we know that the phase PSD consists of all information for PSF characterisation. Therefore, instead of directly modelling the PSF from the focal plane, \cite{Fetick2019} directly parameterises the phase PSD and gives the PSF using Fourier Transform. 

The model seperates the PSD into 2 components: $f <= f_{AO}$ and $f > f_{AO}$. $f_{AO}$ is the AO spatial cutoff frequency, which is the maximum spatial frequency of the phase to be corrected by the AO system.

To get the whole PSD, we need to combine those 2 components such that: 
\begin{equation}
    PSD_{total} = PSD_{AO} + PSD_{halo},
\end{equation}
where $PSD_{AO}$ describes AO-corrected frequencies regions and $PSD_{halo}$ refers to AO-uncorrected frequencies regions. 

The uncorrected area is not modified by the AO system, hence, it follows the Kolmogorov law, 

\begin{equation}
    W_{\phi, Kolmo} (f) = 0.023r_0^{-5/3}f^{-11/3},
\end{equation}

where $f > f_{AO}$ and $r_0$ is the Fried parameter. We call this component $PSD_{halo}$ and it is only set by the knowledge of Fried parameter.

Based on the moffat function, we are able to parameterise the AO-corrected PSD as:

\begin{equation}
    W_\phi(f) = \frac{\beta - 1}{\pi \alpha_x \alpha_y} \frac{M_A(f_x,f_y)}{1-(1+\frac{f_{AO}^2}{\alpha_x \alpha_y})^{1-\beta}}+ C, 
\end{equation}

where $f <= f_{AO}$, (description for parameters). 

\textcolor{red}{Add a section to describe what are the $B_\phi$ and $D_\phi$ in here}

$B_\phi$ is the residual phase co-variance function (?). $D_\phi$ is phase structure function, defined by:
\begin{equation}
    D_\phi (\rho) = \langle (\phi(r) - \phi(r + \rho))^2 \rangle
\end{equation}
Correlation of the phases as a function of spatial distance at a time instance(?). 

As $D_\phi$ increases, it means that the phase has a higher chance of not correlating to the phase we would like to predict or know (or interested in)

For instance, $D_\phi$ increases when $r_0$ decreases , which means it is more difficult for us to predict the phase in other positions.

PSF model is now made of a set of 7 parameters: {$\alpha_x,\alpha_y, \beta, \theta_R, C, r_0, A$}. (description for parameters)

In here, we consider **symmetric** case by setting $\alpha_x = \alpha_y$ and $\theta_R = 0$. Once the PSD and OTF of the telescopes are calculated, we get the PSF using: 
\begin{equation}
    h(\rho / \lambda) = \mathcal{F}^{-1} {h...}.
\end{equation}

In [ ]:
psd_ao = aosys_cls.psd_residual_ao (guess = psf_guess)

# Plotting the PSD of turbulent region (not corrected by the AO)
psd_halo = aosys_cls.psd_residual_halo(r0 = psf_guess[0])

psd = psd_ao + psd_halo


fig, ax =  plt.subplots(1,3, constrained_layout=True, figsize=(20, 10))

ax[0].set_title(r"PSD($\mathrm{f{>}f_{AO}}$)")
pos = ax[0].imshow(np.log10(psd_halo), interpolation='none')
fig.colorbar(pos, ax=ax[0], pad = 0.15, shrink = 0.5)

ax[1].set_title(r"PSD($\mathrm{f{\leq}f_{AO}}$)")
pos1 = ax[1].imshow(np.log10(psd_ao), interpolation='none')
fig.colorbar(pos1, ax=ax[1], pad = 0.15, shrink = 0.5)

ax[2].set_title(r"Total PSD")
pos2 = ax[2].imshow(np.log10(psd_ao+psd_halo), interpolation='none')
fig.colorbar(pos2, ax=ax[2], pad = 0.15,shrink = 0.5)

#### OTF 

In [ ]:
otf_atmo = aosys_cls.otf_atmo(psd)
otf_total = otf_atmo*otf_tel

fig, ax =  plt.subplots(1,3, constrained_layout=True, figsize=(20, 10))

ax[0].set_title(r"$\mathrm{OTF_{atmos}}$")
pos = ax[0].imshow(np.log10(otf_atmo), interpolation='none')
fig.colorbar(pos, ax=ax[0], pad = 0.15, shrink = 0.5)

ax[1].set_title(r"$\mathrm{OTF_{tel}}$")
pos1 = ax[1].imshow(np.real(otf_tel), interpolation='none')
fig.colorbar(pos1, ax=ax[1], pad = 0.15, shrink = 0.5)

ax[2].set_title(r"Total OTF")
pos2 = ax[2].imshow(np.real(otf_total), interpolation='none')
fig.colorbar(pos2, ax=ax[2], pad = 0.15,shrink = 0.5)

Therefore, the PSF would be: 

In [ ]:
psf_total = np.fft.fftshift(np.real(utils.ifft2D(otf_total)))
print(np.min(psf_total))



fig, ax =  plt.subplots(1,2, constrained_layout=True, figsize=(20, 10))
pos = ax[0].imshow(np.log10(psf_total),vmin=-10)
ax[0].set_title('PSF total')
fig.colorbar(pos, ax=ax[0])

pos1 = ax[1].imshow(np.log10(psf_tel),vmin=-10)
ax[1].set_title('PSF telescope')
fig.colorbar(pos1, ax=ax[1])

plt.subplots_adjust(hspace=0.5)
plt.savefig("demo.png", dpi = 300)
plt.show()
# Need to look into - > Calculate the Strehl ratio (ratio of the max of 2 psfs)

In [ ]:
rcParams['figure.figsize'] = 13 ,11
fig, ax = plt.subplots(1,3)
fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)

ycent = int((256*aosys_cls.samp_factor[0])//2)

print(ycent)
ax[0].plot(np.abs(otf_total[ycent,...]))
ax[0].set_title('OTF(total)')
ax[0].axhline(y=1, color = 'r', ls = '--')

ax[1].plot(np.abs(otf_tel)[ycent, :])
ax[1].set_title('OTF(telescope)')
ax[1].axhline(y=1, color = 'r', ls = '--')

ax[2].plot(otf_atmo[ycent, :])
ax[2].set_title('OTF(atmosphere)')
ax[2].axhline(y=1, color = 'r', ls = '--')

print("\nSum of the PSF (which should be excatly 1.)", np.sum(np.abs(psf_total)))
print("\nMax of the otf_atmo: (which should be excatly 1.)", np.max(otf_atmo))
print("\nMax of the otf_tel: (which should be excatly 1.)", np.max(otf_tel))
print("\nMax of the OTF (which should be excatly 1.)", np.max(otf_total))

In [ ]:
guess = 

otf_guess = create_psfao19_otf (otf_tel, guess, aosys_cls)

In [ ]:
rcParams['figure.figsize'] = 13 ,11
fig, ax = plt.subplots()
# fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)

ycent = int((256*aosys_cls.samp_factor[0])//2)

print(ycent)
ax.plot(psf_total[ycent,ycent:512], label = "True PSF parameter")
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_title('PSF')
ax.set_ylabel("Normalisd intensity")
ax.set_ylabel("Pixel")
ax.legend()

How does the shape of the PSF changes with respect to the parameters? 

In [ ]:
r0_list = np.linspace(0.1, 0.9, 10)
sig2_list = np.linspace(0.1, 10, 10)
alpha_list = np.linspace(0.1, 10, 10)

otf_list = [] 
otf_atmo_list = []

In [ ]:
# change r0

otf_list = [] 
otf_atmo_list = []

for i in range (len(r0_list)):
    _psf_guess = psf_guess
    _psf_guess[0] = r0_list[i]
#     _psf_guess[2] = sig2_list[i]
    
    print(_psf_guess)
    
    _otf_atmo, _otf_total = create_psfao19_otf(otf_tel, _psf_guess, aosys_cls)
#     plot_otf_total(aosys_cls,_otf_total )
    otf_list.append(_otf_total)
    otf_atmo_list.append(_otf_atmo)


rcParams['figure.figsize'] = 13 ,11
fig, ax = plt.subplots(1,2)
fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)

ycent = int((256*aosys_cls.samp_factor[0])//2)

print(ycent)
ax[0].plot(np.abs(otf_list[1])[ycent, :], color = 'blue',label = 'r0 = 0.19, sig2 = 1.2' )
ax[0].plot(np.abs(otf_list[9])[ycent, :], color = 'k', ls = '--', label = 'r0 = 0.9, sig2 = 1.2')
ax[0].set_title('OTF(total)')
ax[0].axhline(y=1, color = 'r', ls = '--')
ax[0].legend()

ax[1].plot(np.abs(otf_atmo_list[1])[ycent, :], color = 'blue',label = 'r0 = 0.19, sig2 = 1.2' )
ax[1].plot(np.abs(otf_atmo_list[9])[ycent, :], color = 'k', ls = '--', label = 'r0 = 0.9, sig2 = 1.2')
ax[1].set_title('OTF(atmosphere)')
ax[1].axhline(y=1, color = 'r', ls = '--')
ax[1].legend()

In [ ]:
# change sig2

otf_list = [] 
otf_atmo_list = []

for i in range (len(r0_list)):
    _psf_guess = psf_guess
    _psf_guess[0] = r0_list[0]
    _psf_guess[2] = sig2_list[i]
    
    print(_psf_guess)
    
    _otf_atmo, _otf_total = create_psfao19_otf(otf_tel, _psf_guess, aosys_cls)
#     plot_otf_total(aosys_cls,_otf_total )
    otf_list.append(_otf_total)
    otf_atmo_list.append(_otf_atmo)


rcParams['figure.figsize'] = 13 ,11
fig, ax = plt.subplots(1,2)
fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)

ycent = int((256*aosys_cls.samp_factor[0])//2)

print(ycent)
ax[0].plot(np.abs(otf_list[1])[ycent, :], color = 'blue',label = 'r0 = 0.1, sig2 = 0.1' )
ax[0].plot(np.abs(otf_list[9])[ycent, :], color = 'k', ls = '--', label = 'r0 = 0.9, sig2 = 1.2')
ax[0].set_title('OTF(total)')
ax[0].axhline(y=1, color = 'r', ls = '--')
ax[0].legend()

ax[1].plot(np.abs(otf_atmo_list[1])[ycent, :], color = 'blue',label = 'r0 = 0.1, sig2 = 10' )
ax[1].plot(np.abs(otf_atmo_list[9])[ycent, :], color = 'k', ls = '--', label = 'r0 = 0.9, sig2 = 1.2')
ax[1].set_title('OTF(atmosphere)')
ax[1].axhline(y=1, color = 'r', ls = '--')
ax[1].legend()

In [ ]:
# change alpha

otf_list = [] 
otf_atmo_list = []

for i in range (len(r0_list)):
    _psf_guess = psf_guess
    _psf_guess[0] = r0_list[0]
    _psf_guess[2] = sig2_list[0]
    _psf_guess[3] = alpha_list[i]
    
    print(_psf_guess)
    
    _otf_atmo, _otf_total = create_psfao19_otf(otf_tel, _psf_guess, aosys_cls)
#     plot_otf_total(aosys_cls,_otf_total )
    otf_list.append(_otf_total)
    otf_atmo_list.append(_otf_atmo)


rcParams['figure.figsize'] = 13 ,11
fig, ax = plt.subplots(1,2)
fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)

ycent = int((256*aosys_cls.samp_factor[0])//2)

print(ycent)
ax[0].plot(np.abs(otf_list[0])[ycent, :], color = 'blue',label = 'alpha = 0.1' )
ax[0].plot(np.abs(otf_list[9])[ycent, :], color = 'k', ls = '--', label = 'alpha = 10.' )
ax[0].set_title('OTF(total)')
ax[0].axhline(y=1, color = 'r', ls = '--')
ax[0].legend()

ax[1].plot(np.abs(otf_atmo_list[0])[ycent, :], color = 'blue',label = 'alpha = 0.1' )
ax[1].plot(np.abs(otf_atmo_list[9])[ycent, :], color = 'k', ls = '--', label = 'alpha = 10.')
ax[1].set_title('OTF(atmosphere)')
ax[1].axhline(y=1, color = 'r', ls = '--')
ax[1].legend()

## PSD 

In here, we esitmate the PSD of the object with a model from Conan et al. 1998. This model depends on 3 parameters: $k$, ${\rho} _0$ and $p$. 

\begin{equation}
    S_{obj} = \frac {k} {1 + (f/ \rho_0)^p},
\end{equation}

where $k$ is the value of the object PSD at $f$ = 0 (which is almost the square of the flux), $\rho_0$ is inversely proportional to the characteristic size of the object and $p$ is the decrease power law.  


In [ ]:
k_list = np.linspace(1e17, 9e17, 10)
rho0_list = np.linspace(0.5, 2, 10)
p_list = np.linspace(0.5, 4, 10)
psd_obj_list = []

In [ ]:
# (f/rho0) --> rho
# self.fourier_variable["rho"] = np.fft.fftshift(utils.dist(dimension))/rho0
# self.fourier_variable["psd_object_ini"] = 1./ (np.power(self.fourier_variable["rho"],p) + 1.) # Equation checked


In [ ]:
psd_obj_list = []

# Change k
for i in range(len(k_list)): 
    rho = np.fft.fftshift(utils.dist(256))/rho0_list[0]
    _psd_obj =  k_list[i]/ (np.power(rho,p_list[0]) + 1.)
    
    psd_obj_list.append(_psd_obj)
    
    

fig, ax = plt.subplots(1,1)
fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)
ax.set_xscale('log')
ycent = int((256*aosys_cls.samp_factor[0])//2)
print(ycent)
ax.plot(utils.mean_cir_array(psd_obj_list[0]), color = 'blue',label = 'k = 1e17' )
ax.plot(utils.mean_cir_array(psd_obj_list[9]), color = 'k', ls = '--', label = 'k = 9e17')
ax.set_title('Circular average of the sqaured modulus of PSD (changing k)')
ax.legend()

In [ ]:
# (f/rho0) --> rho
# self.fourier_variable["rho"] = np.fft.fftshift(utils.dist(dimension))/rho0
# self.fourier_variable["psd_object_ini"] = 1./ (np.power(self.fourier_variable["rho"],p) + 1.) # Equation checked

psd_obj_list = []
# Change rho0
for i in range(len(rho0_list)): 
    rho = np.fft.fftshift(utils.dist(512))/rho0_list[i]
    _psd_obj =  k_list[0]/ (np.power(rho,p_list[0]) + 1.)
    
    psd_obj_list.append(_psd_obj)
    

    
fig, ax = plt.subplots(1,1)
fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)

ycent = int((256*aosys_cls.samp_factor[0])//2)
ax.set_xscale('log')
ax.plot(utils.mean_cir_array(psd_obj_list[0]), color = 'blue',label = 'rho0 = 0.5' )
ax.plot(utils.mean_cir_array(psd_obj_list[9]), color = 'k', ls = '--', label = 'rho0 = 2')
ax.set_title('Circular average of the sqaured modulus of PSD (changing rho0)')
ax.legend()

In [ ]:
# (f/rho0) --> rho
# self.fourier_variable["rho"] = np.fft.fftshift(utils.dist(dimension))/rho0
# self.fourier_variable["psd_object_ini"] = 1./ (np.power(self.fourier_variable["rho"],p) + 1.) # Equation checked

psd_obj_list = []

# Change p

for i in range(len(p_list)): 
    rho = np.fft.fftshift(utils.dist(512))/rho0_list[0]
    _psd_obj =  k_list[0]/ (np.power(rho,p_list[i]) + 1.)
    print(p_list[i])
    
    psd_obj_list.append(_psd_obj)
    
fig, ax = plt.subplots(1,1)
fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)

ycent = int((256*aosys_cls.samp_factor[0])//2)
ax.set_xscale('log')
ax.plot(utils.mean_cir_array(psd_obj_list[0]), color = 'blue',label = 'p = 0.5' )
ax.plot(utils.mean_cir_array(psd_obj_list[9]), color = 'k', ls = '--', label = 'p = 4')
ax.set_title('Circular average of the sqaured modulus of PSD (changing p)')
ax.axhline(y=0, color = 'r', ls = '--')
ax.legend()

In [ ]:
np.sum(utils.mean_cir_array(psd_obj_list[0]))

In [ ]:
np.sum(utils.mean_cir_array(psd_obj_list[9]))

In [ ]:
plt.imshow(np.real(np.fft.fft2(psf_tel)))